# Test parsing CSVs to nice plots

This script loads the CSV result files of the STREAM benchmark and creates a polar plot out of it

In [ ]:
# Import required modules
import pandas as pd
import matplotlib.pyplot as plt
import math

In [ ]:
# read in the CSV file
df = pd.read_csv("../source/STREAM/results/STREAM-2-3.csv", index_col=0)
df

In [ ]:
# Define the relevant metrics of the benchmark that should be plotted
metrics = ["Copy", "Add", "Scale", "Triad", "PCIe Read", "PCIe Write"]

# Select the required metrics and the used memory type for further processing
df_relevant = df.T[metrics + ["Memory Type"]]

# Set the PCIe values to 0 for every result that used SVM to prevent weirdly shaped plots
df_relevant.loc[df_relevant["Memory Type"] == "SVM", ["PCIe Read", "PCIe Write"]] = "0.0"

# convert all metrics to float and normalize them
for m in metrics:
    df_relevant[m] = df_relevant[m].astype(float) 
    df_relevant[m] = df_relevant[m] / df_relevant[m].max()
df_relevant

In [ ]:

# Define some colors and line styles that should be used for plotting the data
colors = ["aqua", "royalblue"]
linestyles = ["solid", "dashed", "dashdot", "dotted"]
 
# Calculate the used angle in the polar plot for the different metrics
angles = [n / float(len(df_relevant.T.index) - 1) * 2 * math.pi for n in range(len(df_relevant.T.index) - 1)]
angles += angles[:1]
 
# Initialise the polar plot
fig = plt.figure(figsize=(8,4))
ax = fig.add_subplot(1,1,1, projection="polar")

# Draw one axe per variable and add the labels on x and y axis
plt.xticks(angles[:-1], metrics, color='black', size=11)
plt.yticks([(i + 1) / float(5) for i in range(5)], ["" for i in range(5)])
 
# Iterate over measurements and add them to the plot
for i, dev in enumerate(df_relevant.index):
    row = df_relevant[metrics].T[dev].values.tolist()
    row += row[:1]
    # Plot data and puck the color and line style accordingly using a round-robin scheme
    ax.plot(angles, row, linewidth=1.5, color=colors[i % len(colors)], linestyle=linestyles[int(i / len(colors)) % len(linestyles)], label="%s" % (dev))

# Print the legend outside of the radar plot
ax.legend(loc="center left", bbox_to_anchor=(1.1,0.5))
fig.tight_layout()

# Store the plot as JPEG on the disk
fig.savefig("../source/STREAM/results/stream_results.jpg")